In [1]:
from azul2 import *

In [2]:
import torch
import torch.nn as nn
import torch.quantization as quantization

print(torch.__version__)
print(torch.backends.quantized.supported_engines)

ModuleNotFoundError: No module named 'torch.nn'

In [ ]:
3682 + 8 - 3682 % 8

In [ ]:
class AzulNnue(nn.Module):
    def __init__(self):
        super(AzulNnue, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(3688, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1080),
        )

    def forward(self, x):
        return self.layers(x)

    def get_move(self, game_state):
        possible_moves, _ = game_state.get_possible_moves()
        encoding = encode_game_state(game_state, int(game_state.current_player))
        values = self.forward(torch.tensor(encoding, dtype=torch.float32).clone().detach()).squeeze(0).clone().detach()

        max_value = float('-inf')
        best_move = None
        for move in possible_moves:
            move_index = encode_move(game_state, move)
            value = values[move_index].item()
            if value > max_value:
                max_value = value
                best_move = move

        return best_move

model = AzulNnue()
loss_history = []


In [ ]:
from torch.optim import Adam
import matplotlib.pyplot as plt
from IPython.display import clear_output

loader = DataLoader("http://127.0.0.1:3044", 64)
loader.set_target_buffer_size(3000)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = Adam(model.parameters(), lr=0.001)
# criterion = nn.BCELoss()
criterion = torch.nn.MSELoss(reduction='none')

print("Training started on", device)
step = 0
try:
    sum_loss = 0
    for x, y, y_mask in loader:

        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        y_mask = torch.tensor(y_mask, dtype=torch.float32)

        x = x.to(device)
        y = y.to(device)
        y_mask = y_mask.to(device)

        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        masked_loss = loss * y_mask
        loss = masked_loss.sum() / y_mask.sum()  

        loss.backward()
        optimizer.step()

        step += 1
        # loss_history.append(loss.item())
        sum_loss += loss.item()
        if step % 100 == 0:
            loss_history.append(sum_loss / 100)
            sum_loss = 0

        if step % 1000 == 0:
            clear_output(True)
            plt.plot(loss_history, label='loss')
            plt.legend()
            plt.show()
except KeyboardInterrupt:
    print("Training interrupted")
    model.to("cpu")


In [ ]:
# model = AzulNnue()

# model.qconfig = quantization.get_default_qconfig('fbgemm')

# # Prepare the model for static quantization
# model_prepared = quantization.prepare(model)

# # Example calibration data - this should be representative of the data the model will be used on
# # For simplicity, we are creating random data. Replace this with actual game state encodings.
# calibration_data = [torch.randn(1, 3688) for _ in range(100)]

# # Calibrate the model
# model_prepared.eval()
# with torch.no_grad():
#     for data in calibration_data:
#         model_prepared(data)

# # Convert the model to a quantized version
# model_quantized = quantization.convert(model_prepared)

# # orig = model(x)[0].detach().numpy()
# # quant = model_quantized(x)[0].detach().numpy()
# # orig

# test_data = torch.randn(1, 3688)
# output = model_quantized(test_data)


In [ ]:
loss_history[-1] # 0.0018468010996002704 0.010775361494161188 0.008451616615056992 0.025006712302565574 0.008634282290004194


In [ ]:
import random
model.to("cpu")

game_state = GameState()
game_state.set_player_names(["Neural network", "Random"])
while True:
    possible_moves, state = game_state.get_possible_moves()
    print(game_state)
    if state.is_game_over():
        break
    if int(game_state.current_player) == 0:
        move = model.get_move(game_state)
    else:
        move = random.choice(possible_moves)
    print(move)
    game_state.do_move(move)

game_state

In [ ]:
import time
mcts = MonteCarloTreeSearch()

In [ ]:
# 2 ** 5 * (5 * 2 + 1)
game_state.fen

In [ ]:
# # game_state = GameState("2_1_0_12935366915_47345960200_0-0-0-0-0-100859904_65537000_3_14102-66445_4311876097-17230266369_4345298947-17180000001_1")
# # game_state = GameState("2_0_1_38739051528_4328850691_73744-135169-8194-131329-0-16842752_66126842_0_17078043-3498903_33554688-17180000256_1095250150655-17163223039_0")
# # game_state = GameState("2_0_1_60331723785_16843270_0-0-0-0-0-33555202_65668075_257_16897-268468232_12918456833-17180066304_12968722691-21458191103_1")
# print(game_state.fen)
# print(game_state)
# mcts.stop_working()
# mcts.advance_root(game_state, None)
# mcts.start_working()
# time.sleep(0.7)
# mcts.stop_working()
# # mcts.reset()
# mcts_value = mcts.value
# value_network_evaluation = model.value_of(game_state).detach().item()
# mcts_evaluation = mcts_value[0]

# print("Value network evaluation:", value_network_evaluation)
# print("MCTS evaluation:", mcts_evaluation)

In [ ]:
mcts.principal_variation

In [ ]:
# # encoding = encode_game_state(game_state)
# # tensor_encoding = torch.tensor(encoding, dtype=torch.float32).clone().detach()
# # x = tensor_encoding
# import numpy as np
# x = torch.tensor([0] * 1816, dtype=torch.float32)
# x[0] = 1
# print("Input: ", list(x.clone().detach().numpy()))
# x = model.layers[0](x)
# print("First layer output: ", list(x.clone().detach().numpy()))
# x = model.layers[1](x)
# print("First ReLU output: ", list(x.clone().detach().numpy()))
# x = model.layers[2](x)
# print("Second layer output: ", list(x.clone().detach().numpy()))
# x = model.layers[3](x)
# print("Second ReLU output: ", list(x.clone().detach().numpy()))
# x = model.layers[4](x)
# print("Third layer output: ", list(x.clone().detach().numpy()))
# x = model.layers[5](x)
# print("Sigmoid output: ", list(x.clone().detach().numpy()))

In [ ]:
import json
# state_dict = model.state_dict()
# weights_biases = {k: v.cpu().numpy().tolist() for k, v in state_dict.items()}

# with open('../logs/model_weights.json', 'w') as f:
#     json.dump(weights_biases, f)

# Store the weights binary
# torch.save(model.state_dict(), '../logs/model_weights.pth')
model.to("cpu")
# For each layer extract the weights and biases
layers = []
for layer in model.layers:
    try:
        weights = layer.weight.detach().numpy()
        biases = layer.bias.detach().numpy()
        layers.append(WeightsBiases(weights, biases))
    except Exception as e:
        print(e)
# Store the weights and biases
store_model('../logs/model_weights.bin', layers)

In [ ]:
weights = model.layers[0].get_parameter('weight')
print("weights[0][0]:", weights[0][0].item())
print("weights[0][1]:", weights[0][1].item())
print("weights[1][0]:", weights[1][0].item())

In [ ]:
game_state = GameState("2_0_1_38739051528_4328850691_73744-135169-8194-131329-0-16842752_66126842_0_17078043-3498903_33554688-17180000256_1095250150655-17163223039_0")
print(game_state.fen)
game_state

In [ ]:
encoded = encode_game_state(game_state, 0)
model(torch.tensor(encoded, dtype=torch.float32))

In [ ]:
# Instantiate the model
model = AzulNnue()

# Quantization configuration
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# Fuse layers (if needed, but in this simple case, we have no layers to fuse)
# Fuse operations like Conv + BatchNorm + ReLU into a single operation
# model = torch.quantization.fuse_modules(model, [['layer_name1', 'layer_name2', ...]])

# Prepare the model for static quantization
model = torch.quantization.prepare(model, inplace=True)

# Calibrate the model with representative data
# Here you would normally run a few batches of data through the model
# For example, you could do something like this:
# with torch.no_grad():
#     for data in calibration_data_loader:
#         model(data)

# Convert the model to a quantized version
quant_model = torch.quantization.convert(model, inplace=True)

# Example: Save the quantized model
# torch.save(model.state_dict(), 'azul_nnue_quantized.pth')

# Example: Load the quantized model
# model = AzulNnue()
# model.load_state_dict(torch.load('azul_nnue_quantized.pth'))

# Now, the model is quantized and ready for inference


In [ ]:
def feed_forward_quantized(model, input_data):
    # Ensure input data is a tensor
    if not isinstance(input_data, torch.Tensor):
        input_data = torch.tensor(input_data, dtype=torch.float32)
    # Perform feedforward
    with torch.no_grad():
        output = model(input_data)
    return output

# Example input data (replace with actual data)
input_data = [0.5] * 3688  # Example input of correct size

# Perform feedforward with the quantized model
output = feed_forward_quantized(quant_model, input_data)

print(output)


In [ ]:
GameState("2_0_1_47362607365_25920996362_0-0-0-0-0-65792_65537001_3584_33413-1170_8640331777-17213620480_33750788-4328719615_1")

In [ ]:
71 * 5

In [ ]:
71 ** 2